In [1]:

import os
from obspy import read, Stream
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

# Save and load progress
progress_file = "progress.txt"

def save_progress(file_base, file_index):
    with open(progress_file, "w") as f:
        f.write(f"{file_base},{file_index}")

def load_progress():
    try:
        with open(progress_file, "r") as f:
            content = f.read().strip().split(',')
            return content[0], int(content[1]) if len(content) > 1 else 0
    except FileNotFoundError:
        return None, 0

def plot_st(st, time_shift1, time_shift2):
    sns.set(style="ticks")
    colors = sns.color_palette("Set2", len(st))
    for i, tr in enumerate(st):
        tr.stats.calib = 1.0
        fig = tr.plot(
            starttime=tr.stats.starttime + tr.stats.sac.o - time_shift1,
            endtime=tr.stats.starttime + tr.stats.sac.o + time_shift2,
            color=colors[i],
            handle=True,
            figsize=(6, 5),
        )
        
        ax = fig.gca()
        t_o = tr.stats.starttime + tr.stats.sac.o
        t_p_wave = tr.stats.starttime + tr.stats.sac.t1
        t_s_wave = tr.stats.starttime + tr.stats.sac.t2
        ax.axvline(t_o, color="black", linestyle="-", lw=2, label="Origin time")
        ax.axvline(t_p_wave, color="r", linestyle="--", lw=1.5, label="P-wave")
        ax.axvline(t_s_wave, color="b", linestyle="--", lw=1.5, label="S-wave")
        ax.legend()
        display(fig)

def clear_text_output():
    clear_output(wait=True)
    plt.close('all')

def count_sac_files(directory):
    return len([f for f in os.listdir(directory) if f.endswith(".SAC")])

def process_three_component_data(file_base, source_dir):
    components = ['BHE', 'BHN', 'BHZ']
    st = Stream()
    
    print(f"Attempting to read files for base: {file_base}")
    print(f"Source directory: {source_dir}")
    
    for comp in components:
        file_name = f"{file_base}.{comp}.SAC"
        file_path = os.path.join(source_dir, file_name)
        print(f"Checking for file: {file_path}")
        
        if os.path.exists(file_path):
            try:
                tr = read(file_path)[0]
                st += tr
                print(f"Successfully read {file_name}")
            except Exception as e:
                print(f"Error reading {file_name}: {str(e)}")
        else:
            print(f"Warning: {file_name} not found")
    
    if len(st) != 3:
        print(f"Warning: Not all components found for {file_base}. Found {len(st)} component(s).")
    
    return st

# Main script
source_dir = "./NAFZ_8QC_3SAC"
target_dir = "./NAFZ_9Final_3SAC"

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

sac_file_count = 0
last_processed, last_index = load_progress()
print(f"Last processed file: {last_processed}")
print(f"Last processed index: {last_index}")

start_processing = not last_processed
if start_processing:
    print("Starting from the beginning (no valid progress file found)")
else:
    print(f"Will resume from: {last_processed}")

stop_requested = False

# Updated file base extraction
file_bases = sorted(set(['.'.join(f.split('.')[:-2]) for f in os.listdir(source_dir) if f.endswith(".SAC")]))
total_files = len(file_bases)
print(f"Found {total_files} unique SAC file bases in source directory")

for file_index, file_base in enumerate(file_bases[last_index:], last_index + 1):
    clear_text_output()
    if not start_processing:
        if file_base == last_processed:
            start_processing = True
            print(f"Resuming processing from {file_base}")
        else:
            print(f"Skipping {file_base}")
            continue

    print(f"Processing file base {file_index}/{total_files}: {file_base}")
    try:
        st = process_three_component_data(file_base, source_dir)
    except Exception as e:
        print(f"Error reading files for {file_base}: {e}")
        continue
    
    if len(st) == 3:
        plot_st(st, time_shift1=10, time_shift2=90)
        print(f"Station: {st[0].stats.station}")
        print(f"Distance: {st[0].stats.sac.dist}")
        print(f"Current P-pick: {st[0].stats.sac.t1}, S-pick: {st[0].stats.sac.t2}")
    else:
        print(f"Skipping {file_base} due to missing components")
        continue

    action = input("Enter action (s: save, d: delete, stop: stop processing): ").lower()
    if action == "stop":
        print("Stopping the processing as requested.")
        stop_requested = True
        break
    elif action == "s":
        for tr in st:
            component = tr.stats.channel
            target_path = os.path.join(target_dir, f"{file_base}.{component}.SAC")
            tr.write(target_path, format="SAC")
        sac_file_count += len(st)
        print(f"Files saved for {file_base}")
    elif action == "d":
        print("Files skipped.")
    else:
        print("Invalid action. Files not saved.")
    
    # Save progress
    save_progress(file_base, file_index)
    print(f"Progress saved: {file_base}, Index: {file_index}")

    if stop_requested:
        break

clear_text_output()
if stop_requested:
    print("Processing was stopped by the user.")
else:
    print(
        f"Completed processing and copying SAC files. Total SAC files processed and copied: {sac_file_count}."
    )

# Print the last processed file at the end of the script
last_processed, last_index = load_progress()
print(f"Last processed file at script end: {last_processed}")
print(f"Last processed index at script end: {last_index}")

# Count and print the number of SAC files in the target directory
target_sac_count = count_sac_files(target_dir)
print(f"Number of SAC files in target directory ({target_dir}): {target_sac_count}")


Processing was stopped by the user.
Last processed file at script end: YH.DA01.2012-11-16T12:20
Last processed index at script end: 25
Number of SAC files in target directory (./NAFZ_9Final_3SAC): 60
